# Trabajo Integrador Individual - NLP  
## "De Texto Crudo a Insights: Construyendo un pipeline reproducible de PLN"

> **Nota**: Este cuaderno es una **plantilla lista para usar**. Solo necesitás poner tus archivos `.txt` en la carpeta `corpus/` (una por documento) y completar `metadata.csv` en la raíz con la información de cada archivo.

---

### Estructura esperada
```
/corpus/
  texto_1.txt
  texto_2.txt
  ...
metadata.csv
Trabajo_Integrador_NLP_Base.ipynb
```

### Requisitos técnicos
- Python 3.9+
- Librerías: `pandas`, `numpy`, `scikit-learn`, `nltk`, `spacy`, `gensim`, `pyLDAvis`, `matplotlib`, `wordcloud`
- Modelo spaCy para español (opcional, recomendado): `es_core_news_sm`

> Si no tenés algunas librerías, ejecutá la celda de instalación que sigue.


In [ ]:
import os
import glob
import re
import json
from collections import Counter

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SpanishStemmer

USE_SPACY = True
try:
    import spacy
    try:
        nlp = spacy.load("es_core_news_sm")
    except Exception as e:
        print("No se encontró el modelo 'es_core_news_sm'. Se usará stemming como alternativa.")
        USE_SPACY = False
except Exception as e:
    print("spaCy no disponible. Se usará stemming como alternativa.")
    USE_SPACY = False

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

import gensim
from gensim import corpora, models

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

from wordcloud import WordCloud

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

SPANISH_STOPS = set(stopwords.words('spanish'))
CUSTOM_STOPS = set()


## Sección 1: Presentación del Corpus (15%)

En esta sección, describí **qué corpus elegiste y por qué**. Explicá criterios de selección, tamaño, fuente, y mostrás estadísticas básicas.

**Checklist**:
- [ ] Justificación del dominio elegido
- [ ] Cantidad de documentos y palabras
- [ ] Distribución por categorías/fechas (si aplica)
- [ ] Muestras de texto
- [ ] Nube de palabras (opcional)


In [ ]:
# Cargar metadata
metadata_path = "metadata.csv"
md = pd.read_csv(metadata_path, encoding="utf-8")
print("Registros en metadata:", len(md))
md.head(10)

In [ ]:
# Cargar textos del directorio 'corpus/'
def load_texts_from_folder(folder='corpus'):
    files = sorted(glob.glob(os.path.join(folder, "*.txt")))
    docs = []
    for fp in files:
        with open(fp, "r", encoding="utf-8", errors="ignore") as f:
            text = f.read()
        docs.append({"archivo": os.path.basename(fp), "texto": text})
    return pd.DataFrame(docs)

docs_df = load_texts_from_folder('corpus')
print("Documentos cargados:", len(docs_df))
docs_df.head(5)

In [ ]:
# Unir metadata con textos (por 'archivo')
data = md.merge(docs_df, on='archivo', how='inner')
print("Registros luego del merge:", len(data))

# Completar conteo de palabras si falta
def word_count(txt):
    return len(re.findall(r"\b\w+\b", str(txt), flags=re.UNICODE))

if "palabras" not in data.columns:
    data["palabras"] = data["texto"].apply(word_count)
else:
    data["palabras"] = data["texto"].apply(word_count)

display_cols = [c for c in ["archivo","titulo","autor","fecha","categoria","palabras"] if c in data.columns]
data[display_cols].head(10)

In [ ]:
# Estadística Descriptiva
print("Cantidad de documentos:", len(data))
print("Total de palabras:", int(data['palabras'].sum()))
print("Promedio de palabras por doc:", round(data['palabras'].mean(), 2))

# Distribución por categoría (si existe)
if "categoria" in data.columns:
    print("\nDistribución por categoría:")
    print(data['categoria'].value_counts())

# Distribución temporal (si existe 'fecha')
if "fecha" in data.columns:
    try:
        data['fecha'] = pd.to_datetime(data['fecha'], errors='coerce')
        by_year = data.dropna(subset=['fecha']).groupby(data['fecha'].dt.year).size()
        print("\nDocumentos por año:")
        print(by_year)
    except Exception as e:
        print("No se pudo parsear 'fecha':", e)

In [ ]:
# Muestras de texto
for i, row in data.head(3).iterrows():
    print(f"\n=== {row.get('titulo', row['archivo'])} ===")
    print(str(row['texto'])[:500], '...')

In [ ]:
# Se muestra nube de palabras con todos los textos concatenados
all_text = " ".join(data['texto'].astype(str).tolist()) if len(data) else ""
if all_text.strip():
    wc = WordCloud(width=900, height=500, background_color="white").generate(all_text)
    plt.figure(figsize=(10,5))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title("Nube de Palabras - Corpus completo")
    plt.show()
else:
    print("No hay texto para generar la nube de palabras.")

## Sección 2: Preprocesamiento (20%)

Pasos típicos: minúsculas, limpieza, tokenización, stopwords, lematización/stemming.  
Guardaremos también una **versión limpia** del texto.


In [ ]:
# Funciones de limpieza y normalización
LOWERCASE = True
REMOVE_NUMBERS = False      # activalo si tus números no aportan significado
REMOVE_PUNCT = True
MIN_TOKEN_LEN = 2

# se agrega stopwords propias si tu dominio lo requiere
CUSTOM_STOPS = set([
    # "argentina","años","cosas"
])

PUNCT_RE = re.compile(r"[^\wáéíóúñüÁÉÍÓÚÑÜ#@]+", flags=re.UNICODE)

stemmer = SpanishStemmer()

def simple_tokenize(text):
    # Tokenización simple por palabras
    return nltk.word_tokenize(text, language="spanish")

def normalize_text(text):
    if not isinstance(text, str):
        text = str(text)
    if LOWERCASE:
        text = text.lower()
    if REMOVE_PUNCT:
        text = PUNCT_RE.sub(" ", text)
    if not REMOVE_NUMBERS:
        # se mantiene números (se podria quitar, sino activar REMOVE_NUMBERS)
        pass
    return text

def lemmatize_spacy(tokens):
    if not USE_SPACY:
        return tokens
    doc = nlp(" ".join(tokens))
    return [t.lemma_ for t in doc]

def stem_spanish(tokens):
    return [stemmer.stem(t) for t in tokens]

def remove_stopwords(tokens):
    stops = SPANISH_STOPS.union(CUSTOM_STOPS)
    return [t for t in tokens if t not in stops and len(t) >= MIN_TOKEN_LEN and not t.isspace()]

def preprocess_pipeline(text, use_lemmatize=True):
    text = normalize_text(text)
    toks = simple_tokenize(text)
    toks = remove_stopwords(toks)
    if use_lemmatize and USE_SPACY:
        toks = lemmatize_spacy(toks)
        toks = remove_stopwords(toks)  # quitar stopwords que aparezcan tras lematizar
    else:
        toks = stem_spanish(toks)
    return toks

# Ejecutar el preprocesamiento
data['tokens'] = data['texto'].astype(str).apply(lambda t: preprocess_pipeline(t, use_lemmatize=True))
data['texto_limpio'] = data['tokens'].apply(lambda ts: " ".join(ts))

data[['archivo','palabras','texto_limpio']].head(5)

In [ ]:
# Comparación rápida ex ante y post (longitudes y vocabulario)
orig_wc = data['texto'].apply(lambda t: len(re.findall(r"\b\w+\b", str(t)))).sum()
clean_wc = data['texto_limpio'].apply(lambda t: len(re.findall(r"\b\w+\b", str(t)))).sum()

print("Palabras totales - original:", int(orig_wc))
print("Palabras totales - limpio:", int(clean_wc))

def vocab_size(series):
    vocab = set()
    for txt in series:
        for w in str(txt).split():
            vocab.add(w)
    return len(vocab)

print("Vocabulario original (aprox.):", vocab_size(data['texto']))
print("Vocabulario limpio (aprox.):", vocab_size(data['texto_limpio']))

## Sección 3: Análisis con BoW/TF‑IDF (25%)

- Construcción de matrices documento‑término.
- Palabras más frecuentes / distintivas.
- Similitud entre documentos (coseno).


In [ ]:
# Matriz BoW
bow_vectorizer = CountVectorizer(max_df=0.9, min_df=1)
X_bow = bow_vectorizer.fit_transform(data['texto_limpio'])
bow_terms = np.array(bow_vectorizer.get_feature_names_out())
print("Dimensión BoW:", X_bow.shape)

In [ ]:
# Palabras más frecuentes globalmente (BoW)
term_freq = np.asarray(X_bow.sum(axis=0)).ravel()
top_idx = term_freq.argsort()[::-1][:20]
pd.DataFrame({'termino': bow_terms[top_idx], 'freq': term_freq[top_idx]}).reset_index(drop=True)

In [ ]:
# Matriz TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=1)
X_tfidf = tfidf_vectorizer.fit_transform(data['texto_limpio'])
tfidf_terms = np.array(tfidf_vectorizer.get_feature_names_out())
print("Dimensión TF-IDF:", X_tfidf.shape)

In [ ]:
# Términos con mayor TF-IDF promedio
tfidf_mean = np.asarray(X_tfidf.mean(axis=0)).ravel()
top_tfidf_idx = tfidf_mean.argsort()[::-1][:20]
pd.DataFrame({'termino': tfidf_terms[top_tfidf_idx], 'tfidf_prom': tfidf_mean[top_tfidf_idx]}).reset_index(drop=True)

In [ ]:
# Similitud coseno entre documentos (con TF-IDF)
sim_matrix = cosine_similarity(X_tfidf)
plt.figure(figsize=(6,5))
plt.imshow(sim_matrix, aspect='auto')
plt.title("Similitud coseno entre documentos (TF-IDF)")
plt.colorbar()
plt.xlabel("Documento")
plt.ylabel("Documento")
plt.show()

## Sección 4: Modelado Temático con LDA (20%)

Elegí una cantidad de tópicos **k** (por ejemplo, 3 a 8) y justificá. Mostrá las palabras clave por tópico y documentos representativos. Incluí visualización con `pyLDAvis` si es posible.


In [ ]:
# Preparación para LDA en gensim
texts_for_lda = [t.split() for t in data['texto_limpio'].tolist()]
dictionary = corpora.Dictionary(texts_for_lda)
corpus = [dictionary.doc2bow(text) for text in texts_for_lda]

k = 5  # <-- ajustar la cantidad de tópicos según tu corpus
lda_model = models.LdaModel(corpus=corpus,
                            id2word=dictionary,
                            num_topics=k,
                            random_state=42,
                            update_every=1,
                            chunksize=100,
                            passes=10,
                            alpha='auto',
                            per_word_topics=True)
print(lda_model)

In [ ]:
# Palabras por tópico
topics = lda_model.print_topics(num_topics=k, num_words=10)
for tid, t in topics:
    print(f"\nTópico {tid}: {t}")

In [ ]:
# Visualización con pyLDAvis (puede tardar con muchos docs)
try:
    vis = gensimvis.prepare(lda_model, corpus, dictionary, mds='pcoa')
    pyLDAvis.display(vis)
except Exception as e:
    print("No se pudo generar pyLDAvis:", e)

## Sección 5: Análisis Avanzado (15%)

Elegí **una** alternativa (o más si querés):
1) **Embeddings + reducción de dimensión** (LSA/TruncatedSVD) y visualización.  
2) **Clustering de documentos** (KMeans) sobre TF‑IDF y caracterización de clusters.  
3) **Análisis de sentimiento**



In [ ]:
# (Opción A) LSA sobre TF-IDF + proyección 2D
svd = TruncatedSVD(n_components=2, random_state=42)
X_2d = svd.fit_transform(X_tfidf)

plt.figure(figsize=(6,5))
plt.scatter(X_2d[:,0], X_2d[:,1])
for i, row in data.iterrows():
    plt.text(X_2d[i,0], X_2d[i,1], str(i), fontsize=8)
plt.title("LSA (SVD) sobre TF-IDF - Proyección 2D")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.show()

In [ ]:
# (Opción B) Clustering KMeans sobre TF-IDF
num_clusters = 3  # ajustar según tu corpus
km = KMeans(n_clusters=num_clusters, n_init=10, random_state=42)
labels = km.fit_predict(X_tfidf)

data['cluster'] = labels
print(data[['archivo','cluster']].head(10))

# Palabras top por cluster (aprox. usando centroides)
centroids = km.cluster_centers_
terms = tfidf_terms
for i in range(num_clusters):
    center = centroids[i]
    top_idx = center.argsort()[::-1][:10]
    print(f"\nCluster {i} - términos representativos:", terms[top_idx])

## Sección 6: Conclusiones (5%)

- ¿Qué patrones relevantes encontraste en el corpus?
- ¿Qué técnicas fueron más útiles (BoW/TF‑IDF/LDA/Clustering)?
- Limitaciones del corpus y del pipeline
- Ideas de mejora (ampliar corpus, etiquetas, fine‑tuning, etc.)


## Sección 7: Presentación y Reproducibilidad (≥10%)


